In [13]:
%load_ext autoreload
%autoreload 2
from hillery_classical import Hillery1Party, HillerySecretShare1, make_key, recombine


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Make a secret shared key
Use the convenience function make_key to create a shared key between A, B, C, of length n such that B, C must work together to learn the key. This function assumes that B and C follow the protocol as expected. 

In [14]:
n = 20
A, B, C = [Hillery1Party(x) for x in 'ABC']
make_key(A, B, C, n)
print(A.key)
print(B.key)
print(C.key)

[1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1]
[1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1]
[1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1]


To more accurately represent the real world, and allow for modelling of malicious users, we make each user interact with the secret sharer. The run function checks that all parties have interacted in a valid order, and that the measurement directions are compatible with each other (this should happend about 1/2 of the time if all parties are honest), and runs the secret sharing protocol. Otherwize, it announces the execution failed. Iterating, we can generate the secret one bit at a time.

In [15]:
A, B, C = [Hillery1Party(x) for x in 'ABC']
ss = HillerySecretShare1((A, B, C))

In [16]:
# Run this block multiple times!
for i in range(10):
    A.interact(ss)
    B.interact(ss)
    C.interact(ss) 

    ss.run(verbose=1)
    recombine(B, C)
    print(B.key)

execution failed
[]
execution failed
[]
success
[0]
execution failed
[0]
success
[0, 0]
execution failed
[0, 0]
execution failed
[0, 0]
success
[0, 0, 0]
success
[0, 0, 0, 0]
success
[0, 0, 0, 0, 1]


## Share a quantum state between 2 parties
The second algorithm from Hillery 1998 allows us to share a quantum state between two parties such that they must work together in order to deduce the shared state. In this example, Alice wants to share $$0.6 |0\rangle -0.8i |1\rangle.$$ 
The protocol then requires $B$ to interact with the program, and then depending on $B$'s result, $C$ can perform a correction to the secret qubit to obtain the secret. In the printed wave function $C$'s bit is the third from the right. 

In [17]:
from hillery_quantum import Hillery2Party, HillerySecretShare2, Alice

In [18]:
A = Alice('A', (0.6, -0.8j))
B = Hillery2Party('B')
C = Hillery2Party('C')
ss = HillerySecretShare2((A, B, C))
B.interact(ss)
ss.run(verbose=1)


The resulting wavefunction is  (0.6+0j)|0010> + -0.8j|0110>


## Share a quantum state between many parties
This is the algorithm from Cleve, 1999, which allows a secret to be shared amongst $n$ people such that $k$ of them are required to reconstruct the secret. Here $n$ must be less than or equal to $2k-1$. Due to the fact that this protocol requires that each qubit has $q$ basis states (instead of just 0, 1), we simulate the protocol. In the following example, $n = 5, k = 3$, and the secret is the state $0.8 |1\rangle + 0.6i |3\rangle$

In [19]:
from cleve import CleveParty, CleveSecretShare

In [20]:
parties = [CleveParty(x) for x in range(1, 6)]
p1, p2, p3, p4, p5 = parties
x = CleveSecretShare(parties, 3)
x.split([(1, 4/5), (3, 3/5j)])
print(x.recombine([p1, p4, p5]))


{'001100100100100': 0.8, '001001010001010': 0.8, '001011000011000': 0.8, '001000011000011': 0.8, '001010001010001': 0.8, '001000000000000': 0.8, '001010011010011': 0.8, '001100001100001': 0.8, '001001100001100': 0.8, '001011010011010': 0.8, '001001001001001': 0.8, '001011100011100': 0.8, '001000010000010': 0.8, '001010000010000': 0.8, '001100011100011': 0.8, '001010010010010': 0.8, '001100000100000': 0.8, '001001011001011': 0.8, '001011001011001': 0.8, '001000100000100': 0.8, '001011011011011': 0.8, '001000001000001': 0.8, '001010100010100': 0.8, '001100010100010': 0.8, '001001000001000': 0.8, '011010010010010': -0.6j, '011100000100000': -0.6j, '011001011001011': -0.6j, '011011001011001': -0.6j, '011000100000100': -0.6j, '011011011011011': -0.6j, '011000001000001': -0.6j, '011010100010100': -0.6j, '011100010100010': -0.6j, '011001000001000': -0.6j, '011100100100100': -0.6j, '011001010001010': -0.6j, '011011000011000': -0.6j, '011000011000011': -0.6j, '011010001010001': -0.6j, '01100000